In [1]:
import time
import pandas as pd
import warnings
import feather
warnings.filterwarnings('ignore')
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
pd.set_option('display.max_columns', None)

In [2]:
DF_09 = pd.read_csv('2009/STUDENT.csv')
DF_10 = pd.read_csv('2010/STUDENT.csv')
DF_11 = pd.read_csv('2011/STUDENT.csv')
DF_12 = pd.read_csv('2012/STUDENT.csv')
DF_13 = pd.read_csv('2013/STUDENT.csv')
DF_14 = pd.read_csv('2014/STUDENT.csv')
DF_15 = pd.read_csv('2015/STUDENT.csv')
DF_16 = pd.read_csv('2016/STUDENT.csv')
DF_17 = pd.read_csv('2017/STUDENT.csv')
DF_18 = pd.read_csv('2018/STUDENT.csv')

In [27]:
DF = pd.concat([DF_09,DF_10,DF_11,DF_12,DF_13,DF_14,DF_15,DF_16,DF_17,DF_18])

In [28]:
fields = ['IN_AGUA_INEXISTENTE', 'IN_ESGOTO_INEXISTENTE', 'IN_ENERGIA_INEXISTENTE', 'QT_EQUIP_TV','QT_EQUIP_DVD', 
          'QT_COM_ADMINISTRATIVO','QT_EQUIP_IMPRESSORA','QT_EQUIP_COPIADORA', 'IN_BANDA_LARGA', 'IN_BIBLIOTECA', 
          'IN_LABORATORIO_INFORMATICA', 'IN_LABORATORIO_CIENCIAS',]

DF['IN_INFRA_NENHUMA'] = 1


infra_elementar = (DF.IN_AGUA_INEXISTENTE==0) & (DF.IN_ESGOTO_INEXISTENTE==0)&(DF.IN_ENERGIA_INEXISTENTE ==0)
infra_basica = (DF.QT_EQUIP_TV!=0) & (DF.QT_EQUIP_DVD !=0) & (DF.QT_COM_ADMINISTRATIVO!=0) & (
    DF.QT_EQUIP_IMPRESSORA!=0)

infra_adequada = (DF.QT_EQUIP_COPIADORA != 0)&(DF.IN_BANDA_LARGA != 0)|(DF.IN_BIBLIOTECA!=0)&(
    DF.IN_LABORATORIO_INFORMATICA!=0) 

infra_avancada = (DF.IN_LABORATORIO_CIENCIAS!=0)

DF.loc[(infra_elementar), 'IN_INFRA_ELEMENTAR'] = 1
DF.loc[((infra_elementar) & (infra_basica)), 'IN_INFRA_BASICA'] = 1
DF.loc[((infra_elementar)& (infra_basica) & (infra_adequada)), 'IN_INFRA_ADEQUADA'] = 1
DF.loc[((infra_elementar)& (infra_basica) & (infra_adequada) & (infra_avancada)), 'IN_INFRA_AVANCADA'] = 1

DF.loc[DF['IN_INFRA_ELEMENTAR']==1, 'IN_INFRA_NENHUMA']=0
DF.loc[(DF.IN_INFRA_ELEMENTAR==1) & (DF.IN_INFRA_BASICA==1), 'IN_INFRA_ELEMENTAR'] = 0
DF.loc[(DF.IN_INFRA_BASICA==1) & (DF.IN_INFRA_ADEQUADA==1), 'IN_INFRA_BASICA'] = 0
DF.loc[(DF.IN_INFRA_ADEQUADA==1) & (DF.IN_INFRA_AVANCADA==1), 'IN_INFRA_ADEQUADA'] = 0

DF[['IN_INFRA_ELEMENTAR', 'IN_INFRA_BASICA', 'IN_INFRA_ADEQUADA', 
    'IN_INFRA_AVANCADA']] = DF[['IN_INFRA_ELEMENTAR', 'IN_INFRA_BASICA', 'IN_INFRA_ADEQUADA', 
                                'IN_INFRA_AVANCADA']].fillna(0)

DF['IN_TP_ESCOLA'] = ""
DF.loc[(DF.CO_DEPENDENCIA_ADM== 2) | (DF.CO_DEPENDENCIA_ADM == 3), 'IN_TP_ESCOLA']= 'Municipal+Estadual'
DF.loc[(DF.CO_DEPENDENCIA_ADM == 4), 'IN_TP_ESCOLA'] = 'Privada'
DF.loc[(DF.CO_DEPENDENCIA_ADM == 1), 'IN_TP_ESCOLA'] = 'Federal'

In [29]:
enem = [
'CO_ANO',
'CO_ESCOLA',
'CO_UF',
'CO_MUNICIPIO',
'TP_ENSINO',
'NU_IDADE',
'TP_SEXO',
'TP_COR_RACA',
'TP_ST_CONCLUSAO',
'NU_NOTA_CN',
'NU_NOTA_CH',
'NU_NOTA_LC',
'NU_NOTA_MT',
'NU_NOTA_REDACAO',
'EDU_PAI',
'EDU_MAE',
'QT_PESSOAS_CASA',
'RENDA_MENSAL',
'CO_DEPENDENCIA_ADM',
'NU_NOTA_GERAL', 
'TARGET',
'IN_TP_ESCOLA',
'IN_INFRA_ELEMENTAR',
'IN_INFRA_BASICA', 
'IN_INFRA_ADEQUADA', 
'IN_INFRA_AVANCADA',
'IN_INFRA_NENHUMA'
    
]
DF = DF.filter(items = enem )

In [30]:
k= pd.DataFrame()
k['miss'] = DF.isnull().sum()/DF.shape[0]
k['type'] = DF.dtypes
k.T

,CO_ANO,CO_ESCOLA,CO_UF,CO_MUNICIPIO,NU_IDADE,TP_SEXO,TP_COR_RACA,TP_ST_CONCLUSAO,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,NU_NOTA_REDACAO,EDU_PAI,EDU_MAE,QT_PESSOAS_CASA,RENDA_MENSAL,CO_DEPENDENCIA_ADM,NU_NOTA_GERAL,IN_TP_ESCOLA,IN_INFRA_ELEMENTAR,IN_INFRA_BASICA,IN_INFRA_ADEQUADA,IN_INFRA_AVANCADA,IN_INFRA_NENHUMA
miss,0,0,0,0,0,0,0,0.0964968,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
type,int64,float64,int64,int64,float64,int64,float64,float64,float64,float64,float64,float64,float64,int64,int64,float64,int64,int64,float64,object,float64,float64,float64,float64,int64


In [31]:
#by context, lets fill it with 0
DF.fillna(0, inplace=True,)

In [32]:
DF.CO_DEPENDENCIA_ADM.value_counts()

2    7515288
4    2411583
1     251243
3      97007
Name: CO_DEPENDENCIA_ADM, dtype: int64

In [33]:
DF.loc[(DF.CO_DEPENDENCIA_ADM== 2) | (DF.CO_DEPENDENCIA_ADM == 3), 'CO_DEPENDENCIA_ADM']= 'Municipal+Estadual'
DF.loc[(DF.CO_DEPENDENCIA_ADM == 4), 'CO_DEPENDENCIA_ADM'] = 'Privada'
DF.loc[(DF.CO_DEPENDENCIA_ADM == 1), 'CO_DEPENDENCIA_ADM'] = 'Federal'


DF.loc[(DF.TP_COR_RACA == 0), 'TP_COR_RACA'] = 'Não declarada'
DF.loc[(DF.TP_COR_RACA == 1), 'TP_COR_RACA'] = 'Branca'
DF.loc[(DF.TP_COR_RACA == 2), 'TP_COR_RACA'] = 'Preta'
DF.loc[(DF.TP_COR_RACA == 3), 'TP_COR_RACA'] = 'Parda'
DF.loc[(DF.TP_COR_RACA == 4), 'TP_COR_RACA'] = 'Amarela'
DF.loc[(DF.TP_COR_RACA == 5), 'TP_COR_RACA'] = 'Indígena'

DF.loc[(DF.TP_SEXO == 0), 'TP_SEXO'] = 'Feminino'
DF.loc[(DF.TP_SEXO == 1), 'TP_SEXO'] = 'Masculino'

DF.loc[(DF.EDU_PAI == 0), 'EDU_PAI'] = '< 4ªSérie/Não sei'
DF.loc[(DF.EDU_PAI == 1), 'EDU_PAI'] = '4ª Série'
DF.loc[(DF.EDU_PAI == 2), 'EDU_PAI'] = '8ª Série'
DF.loc[(DF.EDU_PAI == 3), 'EDU_PAI'] = 'Ens. Médio'
DF.loc[(DF.EDU_PAI == 4), 'EDU_PAI'] = 'Faculdade'
DF.loc[(DF.EDU_PAI == 5), 'EDU_PAI'] = 'Pós-Gradução'


DF.loc[(DF.EDU_MAE == 0), 'EDU_MAE'] = '< 4ªSérie/Não sei'
DF.loc[(DF.EDU_MAE == 1), 'EDU_MAE'] = '4ª Série'
DF.loc[(DF.EDU_MAE == 2), 'EDU_MAE'] = '8ª Série'
DF.loc[(DF.EDU_MAE == 3), 'EDU_MAE'] = 'Ens. Médio'
DF.loc[(DF.EDU_MAE == 4), 'EDU_MAE'] = 'Faculdade'
DF.loc[(DF.EDU_MAE == 5), 'EDU_MAE'] = 'Pós-Gradução'


DF.loc[(DF.RENDA_MENSAL == 0), 'RENDA_MENSAL'] = 'Nenhuma'
DF.loc[(DF.RENDA_MENSAL == 1), 'RENDA_MENSAL'] = 'Até 1 sal.'
DF.loc[(DF.RENDA_MENSAL == 2), 'RENDA_MENSAL'] = '1 a 2 sal.'
DF.loc[(DF.RENDA_MENSAL == 3), 'RENDA_MENSAL'] = '2 a 5 sal.'
DF.loc[(DF.RENDA_MENSAL == 4), 'RENDA_MENSAL'] = '5 a 10 sal.'
DF.loc[(DF.RENDA_MENSAL == 5), 'RENDA_MENSAL'] = 'Acima de 10 sal.'



In [34]:
DF['STD_MEAN'] = DF.groupby(['CO_ANO','CO_ESCOLA'])['NU_NOTA_GERAL'].transform('std')

In [35]:
#renda percapita based 2020

DF.loc[DF['RENDA_MENSAL']=='Nenhuma', 'RENDA_NUM']=0
DF.loc[DF['RENDA_MENSAL']=='Até 1 sal.', 'RENDA_NUM']= 1039/2
DF.loc[DF['RENDA_MENSAL']=='1 a 2 sal.', 'RENDA_NUM']= ((1039*2 - 1039)/2)+1039
DF.loc[DF['RENDA_MENSAL']=='2 a 5 sal.', 'RENDA_NUM']=((1039*5 - 1039*2)/2)+1039*2
DF.loc[DF['RENDA_MENSAL']=='5 a 10 sal.', 'RENDA_NUM']=((1039*10 - 1039*5)/2)+1039*5
DF.loc[DF['RENDA_MENSAL']=='Acima de 10 sal.', 'RENDA_NUM']=1039*10

DF['RENDA_PERCAPITA'] = DF.RENDA_NUM/DF.QT_PESSOAS_CASA



In [37]:
DF.columns

Index(['CO_ANO', 'CO_ESCOLA', 'CO_UF', 'CO_MUNICIPIO', 'NU_IDADE', 'TP_SEXO',
       'TP_COR_RACA', 'TP_ST_CONCLUSAO', 'NU_NOTA_CN', 'NU_NOTA_CH',
       'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_REDACAO', 'EDU_PAI', 'EDU_MAE',
       'QT_PESSOAS_CASA', 'RENDA_MENSAL', 'CO_DEPENDENCIA_ADM',
       'NU_NOTA_GERAL', 'IN_TP_ESCOLA', 'IN_INFRA_ELEMENTAR',
       'IN_INFRA_BASICA', 'IN_INFRA_ADEQUADA', 'IN_INFRA_AVANCADA',
       'IN_INFRA_NENHUMA', 'STD_MEAN', 'RENDA_NUM', 'RENDA_PERCAPITA'],
      dtype='object')

In [36]:
DF.to_csv('ALL_STUDENTS.csv', index =False)